In [8]:
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

ModuleNotFoundError: No module named 'numpy'

In [7]:
client = weaviate.connect_to_local(
    host="localhost",
    port=8080
)

WeaviateConnectionError: Connection to Weaviate failed. Details: Error: [Errno 61] Connection refused. 
Is Weaviate running and reachable at http://localhost:8080?